# 예제 3.58: 단층 퍼셉트론 (Single Layer Perceptron)

## 학습목표
1. **퍼셉트론(Perceptron)** 개념 이해하기 - 가장 기본적인 신경망 구조
2. **단층 퍼셉트론의 한계** 파악하기 - XOR 문제 해결 불가
3. **선형 분리 가능(Linearly Separable)** 개념 이해하기
4. **AND, OR 문제** 학습 가능 확인하기

---

#### 라이브러리 및 클래스 정의

**단층 퍼셉트론**
- 입력층과 출력층만 존재 (은닉층 없음)
- 선형 분리 가능한 문제만 해결 가능
- AND, OR는 학습 가능, XOR는 학습 불가

In [ ]:
import torch
import pandas as pd
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader


class CustomDataset(Dataset):
    """퍼셉트론용 데이터셋 (2입력 → 1출력)"""
    
    def __init__(self, file_path):
        df = pd.read_csv(file_path)
        self.x1 = df.iloc[:, 0].values  # 첫 번째 입력
        self.x2 = df.iloc[:, 1].values  # 두 번째 입력
        self.y = df.iloc[:, 2].values   # 이진 레이블
        self.length = len(df)

    def __getitem__(self, index):
        x = torch.FloatTensor([self.x1[index], self.x2[index]])
        y = torch.FloatTensor([self.y[index]])
        return x, y

    def __len__(self):
        return self.length


class CustomModel(nn.Module):
    """단층 퍼셉트론 모델"""
    
    def __init__(self):
        super().__init__()
        # 단층: 입력 2 → 출력 1 (은닉층 없음)
        self.layer = nn.Sequential(
            nn.Linear(2, 1),  # 선형 변환
            nn.Sigmoid()       # 활성화 함수
        )

    def forward(self, x):
        x = self.layer(x)
        return x

---

#### 데이터 및 모델 준비

In [ ]:
# 데이터셋 및 데이터로더 생성
# perceptron.csv: AND 또는 OR 논리 게이트 데이터
train_dataset = CustomDataset("../datasets/perceptron.csv")
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True, drop_last=True)

In [ ]:
# GPU 설정
device = "cuda" if torch.cuda.is_available() else "cpu"

# 단층 퍼셉트론 모델 생성
model = CustomModel().to(device)

# 손실함수 및 옵티마이저
criterion = nn.BCELoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01)

---

#### 학습 루프

In [ ]:
# 학습 루프
for epoch in range(10000):
    cost = 0.0

    for x, y in train_dataloader:
        x = x.to(device)
        y = y.to(device)

        output = model(x)
        loss = criterion(output, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        cost += loss

    cost = cost / len(train_dataloader)

    if (epoch + 1) % 1000 == 0:
        print(f"Epoch : {epoch+1:4d}, Cost : {cost:.3f}")

---

#### 추론 (논리 게이트 테스트)

**테스트 입력**: (0,0), (0,1), (1,0), (1,1)

AND 게이트라면: 0, 0, 0, 1
OR 게이트라면: 0, 1, 1, 1

In [ ]:
# 추론: 모든 입력 조합 테스트
with torch.no_grad():
    model.eval()
    
    # 테스트 입력: 가능한 모든 이진 조합
    inputs = torch.FloatTensor([
        [0, 0],  # 입력 1
        [0, 1],  # 입력 2
        [1, 0],  # 입력 3
        [1, 1]   # 입력 4
    ]).to(device)
    
    outputs = model(inputs)
    
    print("단층 퍼셉트론 결과:")
    print("---------")
    print("예측 확률:")
    print(outputs)
    print("\n예측 클래스 (임계값 0.5):")
    print((outputs > 0.5).int())